In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dm4bem

### Introduction

In this first part we will define our working environment. The plan of our house can be found below. The objective of this study case is the nalysis of a cubic building with 5 identical walls & a transparent wall (glass window), air infiltration, and HVAC system controlling the indoor air temperature.

### Design model

The room that we will study is presented in the figure below. Two rooms are separated by a small wall without insulation. The total area of the model is about 9 meters square and the ceiling is 3 meters high. The rooms have each 3 exterior walls and one interior wall. We will consider the house as insulated from the outside .
The interior walls are made of concrete. The room also has a 1 m² window, a 2 m² door and a HVAC device.



<center><img src="https://hub.gke2.mybinder.org/user/cghiaus-dm4bem-v9wqwu31/files/Adobe%20Scan%2024%20avr.%202023%20(1)%20(1).pdf?_xsrf=2%7C8d3810e3%7C68e4a5cde0ead2325dbafcff04581392%7C1682336278"/></center>

> Figure 1. Plan of the building



We will now define boundary conditions wich are important to solve our case.


### Hypothesis

Boundary Conditions:

- Temperature of the exterior air = 0 °C
- Setpoint temperature of the interior air = 18 °C
- Unique temperature on each wall
- Unidirectionnal heat transfer

### Dimension of windows, doors, and walls.

In [2]:
l = 3               # m length of the cubic room
Sg = l**2           # m² surface of the glass wall
Sc = Si = 5 * Sg    # m² surface of concrete & insulation of the 5 walls

Thermo-physical properties
The thermophysical properties of the air (in SI units) are:

In [3]:
air = {'Density': 1.2,                      # kg/m³
       'Specific heat': 1000}               # J/(kg·K)
# pd.DataFrame.from_dict(air, orient='index', columns=['air'])
pd.DataFrame(air, index=['Air'])

,Density,Specific heat
Air,1.2,1000


We use concrete, glass and insulation (glass wool) as materials. We take our door with oak wood. 

In [4]:
BETON = {'Conductivity': 1.400,
            'Density': 2400.0,
            'Specific heat': 880,
            'Width': 0.2,
            'Surface': 60}

ISOLANT = {'Conductivity': 0.046,
              'Density': 25.0,
              'Specific heat': 1030,
              'Width': 0.08,
              'Surface':45}

VERRE = {'Conductivity': 1.4,
         'Density': 2500,
         'Specific heat': 1210,
         'Width': 0.04,
         'Surface': 40}

PORTE = {'Conductivity': 0.15,
         'Density': 800,
         'Specific heat': 2000,
         'Width': 0.1,
         'Surface': 4}

wall = pd.DataFrame.from_dict({'Layer_out': ISOLANT,
                               'Layer_in': BETON,
                               'Glass': VERRE,
                               'Door': PORTE},
                              orient='index')
wall

,Conductivity,Density,Specific heat,Width,Surface
Layer_out,0.046,25.0,1030,0.08,45
Layer_in,1.400,2400.0,880,0.20,60
Glass,1.400,2500.0,1210,0.04,40
Door,0.150,800.0,2000,0.10,4


### Convection coefficients

Conventional values for the [convection coeficients](https://energieplus-lesite.be/theories/enveloppe9/echanges-chaleur-parois/resistance-thermique-d-echange-superficiel/) for indoor and outdoor convection in W/(m²⋅K) are:

In [5]:
h = pd.DataFrame([{'in': 8., 'out': 25}], index=['h'])  # W/(m²⋅K)
# h = pd.DataFrame([{'in': 8., 'out': 25}])  # W/(m²⋅K)
h

,in,out
h,8.0,25


### Thermal coductances
#### Conduction
The conductances 1, 2, 3, and 4 of the thermal circuit from Figure 3 model the heat transfer by [conduction](https://en.m.wikipedia.org/wiki/Thermal_conduction). Conduction conductances, in W/K, are of the form:
$$G_{cd} = \frac{\lambda}{w}S$$
where:

- $\lambda$ - [thermal conductvity](https://en.m.wikipedia.org/wiki/Thermal_conductivity), W/(m⋅K);
- $w$ - width of the material, m;
- $S$ - surface area of the wall, m².

In [6]:
# conduction
G_cd = wall['Conductivity'] / wall['Width'] * wall['Surface']
pd.DataFrame(G_cd, columns={'Conductance'})

,Conductance
Layer_out,25.875
Layer_in,420.000
Glass,1400.000
Door,6.000


### Partie codage 

Création du code 

$$\left\{\begin{array}{ll}
C \dot{\theta} = -(A^T G A) \theta + A^T G b + f\\ 
q = G (-A \theta + b)
\end{array}\right.$$